In [ ]:
!pip install openai==0.28 # openai 라이브러리를 설치합니다.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 714.5 kB/s eta 0:00:00


In [ ]:
import openai
from google.colab import userdata

openai.api_key = userdata.get('OPENAI_API_KEY')

In [ ]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

functions = [
    {
        "name": "get_current_weather", # 함수 이름이 명확해야 합니다.
        "description": "Get the current weather in a given location", # 설명도 상세하게 작성해야 합니다.
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string", # 타입
                    "description": "The city and state, e.g. San Francisco, CA", # 설명
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

In [ ]:
def llm(input_text, chat_history):

    if len(chat_history) == 0:
        chat_history.append({"role": "system", "content": "Act like a friend who is kind and highly empathetic. Respond to the user's input in a friendly and conversational manner in Korean"})

    chat_history.append({"role": "user", "content": input_text})

    print(chat_history)

    response = openai.ChatCompletion.create(model="gpt-3.5-turbo-0613",
                                              messages = chat_history,
                                              functions = functions, # 편지봉투에 편지지 뿐만 아니라 함수 설명서도 동봉하자
                                              function_call = "auto",  # auto is default, but we'll be explicit
                                              )

    response_message = response["choices"][0]["message"]

    # function_call vs contents

    # Step 2: check if GPT wanted to call a function
    if response_message.get("function_call"):
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors

        available_functions = {
            "get_current_weather": get_current_weather,
        }  # only one function in this example, but you can have multiple

        function_name = response_message["function_call"]["name"]

        # 함수 선정
        function_to_call = available_functions[function_name]

        # 함수 인자 가져오기
        function_args = json.loads(response_message["function_call"]["arguments"])

        # AI가 어떤 함수를 선택했고, 어떤 인자를 입력했는 지
        chat_history.append(response_message)  # extend conversation with assistant's reply

        # 호출
        function_response = function_to_call(**function_args)

        # role function에 함수 결과를 넘긴다.

        # Step 4: send the info on the function call and function response to GPT
        chat_history.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )

        # extend conversation with function response
        # 함수 호출 결과를 사용자 지시에 맞게 잘 설명해주는 용도로 호출
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=chat_history,
        )

        output = second_response.choices[0].message.content

    else:
        output = response.choices[0].message.content

    chat_history.append({"role": "assistant", "content": output})

    return output

In [ ]:
def chat_with_user(user_message, chat_history):
    ai_message = llm(user_message, chat_history)
    return ai_message

chat_history = []

while True:
    user_message = input("USER > ")
    if user_message.lower() == "quit":
        break
    ai_message = chat_with_user(user_message, chat_history)
    print(f" A I > {ai_message}")

USER > quit
